 if first_card_container:
              card1=card.find('div', class_='mb-srp__card')
               if card1:mb-srp__card


This notebook is created in Google Colab and is designed to scrape property rate trends from the **Magic Bricks** website. The data focuses on the **prices** and **quarterly growth** of **residential properties** in **Chennai.

**

**Key Features of the Code:**

**1. Data Scraping:**

* The code extracts data from the Magic Bricks website using the requests and BeautifulSoup libraries.
* It captures detailed information, including property prices and their growth trends, by parsing tables from the webpage.

**2. Handling Multiple Pages:**

* The code efficiently handles data from multiple pages of the website.
* A loop iterates through the pages to combine the data into a single comprehensive dataset.
**3. Data Aggregation:**
* The scraped data from each page is stored in a Pandas DataFrame.
* All the data is merged into a single DataFrame (combined_df) for seamless analysis.

Installing the Python libraries using the pip package manager:

In [2]:
! pip install requests
! pip install beautifulsoup4
! pip install pandas
! pip install selenium


Importing the necessary libraries

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

This code scrapes property rate trends from a specified webpage, processes the table data, and returns it in a structured Pandas DataFrame format for analysis.

In [49]:
def fetch_webpage(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'  # Explicitly set encoding to UTF-8
        if response.status_code == 200:
            print(f"Webpage fetched successfully: {url}")
            return response.content
        else:
            print(f"Failed to fetch the webpage. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching webpage: {e}")
        return None

In [50]:
def scrape_page(url):
    all_card_data = []
    page = fetch_webpage(url)
    if page:
        soup = BeautifulSoup(page, "html.parser")
        main_cards = soup.find_all('div', class_='mb-srp__card')


        for card in main_cards:
            card_data = {}  # Dictionary to store data for this card

            # First sub-card with property details
            first_card_container = card.find('div', class_='mb-srp__card__container')

            if first_card_container:
                target_sub_card = first_card_container.find('div', class_='mb-srp__card__info')
                if target_sub_card:
                    property_name = target_sub_card.find('h2', class_='mb-srp__card--title').get_text(strip=True)
                    card_data['Property Name'] = property_name
                    details_card = target_sub_card.find('div', class_='mb-srp__card__summary')
                    if details_card:
                        list_card = details_card.find('div', class_='mb-srp__card__summary__list')
                        if list_card:
                            list_items = list_card.find_all('div', class_='mb-srp__card__summary__list--item')
                            for item in list_items:
                                label_div = item.find('div', class_='mb-srp__card__summary--label')
                                value_div = item.find('div', class_='mb-srp__card__summary--value')
                                if label_div and value_div:
                                    column_name = item.get('data-summary', label_div.get_text(strip=True))
                                    value = value_div.get_text(strip=True)
                                    card_data[column_name] = value


            # Second sub-card with total price and price per sqft
            second_card = card.find('div', class_='mb-srp__card__estimate')
            if second_card:
                total_price = second_card.find('div', class_='mb-srp__card__price--amount').get_text(strip=True)
                card_data['Total Price'] = total_price

            all_card_data.append(card_data)
    return all_card_data

In [51]:
page_num=1
base_url = "https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr"
url = f"{base_url}/page-{page_num}"
print(f"Scraping page {page_num}...")
page_data = scrape_page(url)
print(page_data)

Scraping page 1...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-1
[{'Property Name': '3 BHK House for Rent in PJR Layout, Hyderabad', 'furnishing': 'Semi-Furnished', 'bathroom': '4', 'tenent-preffered': 'Family', 'status': 'Immediately', 'carpet-area': '2653 sqft', 'facing': 'West', 'overlooking': 'Garden/Park', 'balcony': '2', 'Total Price': '₹83,000'}, {'Property Name': '1 BHK House for Rent in Uppal Ramanthapur Road, Hyderabad', 'furnishing': 'Semi-Furnished', 'bathroom': '1', 'tenent-preffered': 'Bachelors/Family', 'status': 'Immediately', 'carpet-area': '420 sqft', 'floor': '1 out of 1', 'facing': 'North', 'overlooking': 'Main Road', 'balcony': '1', 'Total Price': '₹9,000'}, {'Property Name': '2 BHK House for Rent in Ameenpur, Hyderabad', 'furnishing': 'Semi-Furnished', 'bathroom': '2', 'tenent-preffered': 'Bachelors', 'status': 'Immediately', 'carpet-area': '1000 sqft', 'floor': 'Ground out of 2', 'facing': 'East', '

In [52]:
def scrape_multiple_pages(base_url,numpages):
    all_data = []
    for page_num in range(1, num_pages + 1):
        url = f"{base_url}/page-{page_num}"
        print(f"Scraping page {page_num}...")
        page_data = scrape_page(url)
        all_data.extend(page_data)
    return all_data

In [53]:
# Base URL of the website (without page number)
base_url = "https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr"
num_pages = 50  # Set the number of pages to scrape

# Scrape data from multiple pages
all_data = scrape_multiple_pages(base_url, num_pages)

# Print the collected data (or you can save it to a file)
print(f"Collected data from {len(all_data)} properties.")

Scraping page 1...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-1
Scraping page 2...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-2
Scraping page 3...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-3
Scraping page 4...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-4
Scraping page 5...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-5
Scraping page 6...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-6
Scraping page 7...
Webpage fetched successfully: https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr/page-7
Scraping page 8...
Webpage fetched successfully: https://www.magicbricks.com/independent-h

In [54]:
#all_data

In [55]:
#Create a DataFrame
df = pd.DataFrame(all_data)

# Display the DataFrame
df

# Save to CSV (optional)
df.to_csv('properties_data.csv', index=False)


In [56]:

'''page_num=1
base_url = "https://www.magicbricks.com/independent-house-for-rent-in-new-delhi-pppfr"
url = f"{base_url}/page-{page_num}"
print(f"Scraping page {page_num}...")
page_data = scrape_page(url)
print(page_data)'''

'page_num=1\nbase_url = "https://www.magicbricks.com/independent-house-for-rent-in-new-delhi-pppfr"\nurl = f"{base_url}/page-{page_num}"\nprint(f"Scraping page {page_num}...")\npage_data = scrape_page(url)\nprint(page_data)'

In [58]:
'''
URL for different metro cities
https://www.magicbricks.com/independent-house-for-rent-in-new-delhi-pppfr
https://www.magicbricks.com/independent-house-for-rent-in-banglore-pppfr
https://www.magicbricks.com/independent-house-for-rent-in-pune-pppfr
https://www.magicbricks.com/independent-house-for-rent-in-mumbai-pppfr
https://www.magicbricks.com/independent-house-for-rent-in-kolkata-pppfr
https://www.magicbricks.com/independent-house-for-rent-in-chennai-pppfr
https://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr
'''

'\nURL for different metro cities\nhttps://www.magicbricks.com/independent-house-for-rent-in-new-delhi-pppfr\nhttps://www.magicbricks.com/independent-house-for-rent-in-banglore-pppfr\nhttps://www.magicbricks.com/independent-house-for-rent-in-pune-pppfr\nhttps://www.magicbricks.com/independent-house-for-rent-in-mumbai-pppfr\nhttps://www.magicbricks.com/independent-house-for-rent-in-kolkata-pppfr\nhttps://www.magicbricks.com/independent-house-for-rent-in-chennai-pppfr\nhttps://www.magicbricks.com/independent-house-for-rent-in-hyderabad-pppfr\n'